# ABM Example 2: The spread of SARS-CoV-2 (Graph model)

[Source](https://juliadynamics.github.io/Agents.jl/stable/examples/sir/) from Agents.jl tutorial

Here we add one more category of individuals: those who are infected, but do not know it. Transmission rate for infected and diagnosed individuals is lower than infected and undetected. 

In [4]:
using Agents
using Plots
using Random
using DataFrames
using LightGraphs
using Distributions
using GraphRecipes
using LinearAlgebra: diagind

In [5]:
mutable struct PoorSoul <: AbstractAgent
    id::Int
    pos::Int            # Which city
    days_infected::Int  # number of days since is infected
    status::Symbol      # 1: S, 2: I, 3:R
end

In [6]:
function make_SIRgraph(;
    Ns,
    migration_rates,
    β_und,
    β_det,
    infection_period = 30,
    reinfection_probability = 0.05,
    detection_time = 14,
    death_rate = 0.02,
    Is = [zeros(Int, length(Ns) - 1)..., 1],
    seed = 0,
)

    rng = MersenneTwister(seed)
    @assert length(Ns) ==
    length(Is) ==
    length(β_und) ==
    length(β_det) ==
    size(migration_rates, 1) "length of Ns, Is, and B, and number of rows/columns in migration_rates should be the same "
    @assert size(migration_rates, 1) == size(migration_rates, 2) "migration_rates rates should be a square matrix"

    C = length(Ns) # Number of cities
	
    # normalize migration_rates
    migration_rates_sum = sum(migration_rates, dims = 2)
    for c in 1:C
        migration_rates[c, :] ./= migration_rates_sum[c]
    end

    properties = (;
        Ns,
        Is,
        β_und,
        β_det,
        migration_rates,
        infection_period,
        reinfection_probability,
        detection_time,
        C,
        death_rate
    )
	
	
    space = GraphSpace(complete_digraph(C))
    model = ABM(PoorSoul, space; properties, rng)

    # Add initial individuals
    for city in 1:C, n in 1:Ns[city]
        ind = add_agent!(city, model, 0, :S) # Susceptible
    end
    # add infected individuals
    for city in 1:C
        inds = ids_in_position(city, model)
        for n in 1:Is[city]
            agent = model[inds[n]]
            agent.status = :I # Infected
            agent.days_infected = 1
        end
    end
    return model
end

make_SIRgraph (generic function with 1 method)

In [7]:
function make_SIRgraphParams(;
	C,
    max_travel_rate,
    infection_period = 30,
    reinfection_probability = 0.05,
    detection_time = 14,
    death_rate = 0.02,
    Is = [zeros(Int, C - 1)..., 1],
    seed = 19,
)
	# For reproducibility
	Random.seed!(seed)
	
	# City population
    Ns = rand(50:5000, C)
	
	# Undetected transmission
    β_und = rand(0.3:0.02:0.6, C)
	
	# Detected transmission (10% of undetected)
    β_det = β_und ./ 10
	
	# Migrate from city i to city j
	# People in small cities tend to migrate to bigger cities
	migration_rates = zeros(C, C)
    for c in 1:C, c2 in 1:C
        migration_rates[c, c2] = (Ns[c] + Ns[c2]) / Ns[c]
    end
	
	# Normalize migration rates
	maxM = maximum(migration_rates)
    migration_rates = (migration_rates .* max_travel_rate) ./ maxM
	
	# Migrate to self = 1
    migration_rates[diagind(migration_rates)] .= 1.0
	
	return (; Ns,
        β_und,
        β_det,
        migration_rates,
        infection_period,
        reinfection_probability,
        detection_time,
        death_rate,
        Is)
end

make_SIRgraphParams (generic function with 1 method)

In [8]:
SIRgraphparams = make_SIRgraphParams(C = 8, max_travel_rate = 0.01)

(Ns = [4148, 1355, 4929, 160, 3079, 77, 3729, 2043], β_und = [0.34, 0.36, 0.5, 0.36, 0.4, 0.48, 0.42, 0.48], β_det = [0.034, 0.036, 0.05, 0.036, 0.04, 0.048, 0.041999999999999996, 0.048], migration_rates = [1.0 0.00020406129809127796 … 0.0002920935571624562 0.00022957359558115601; 0.0006246835900240745 1.0 … 0.0005771200021229137 0.00038573048135595215; … ; 0.00032491393808256055 0.0002097070536005761 … 1.0 0.00023808597824203875; 0.00046611418231553366 0.0002558320128425429 … 0.0004345680924447198 1.0], infection_period = 30, reinfection_probability = 0.05, detection_time = 14, death_rate = 0.02, Is = [0, 0, 0, 0, 0, 0, 0, 1])

In [9]:
function migrate!(agent, model)
    pid = agent.pos
    d = DiscreteNonParametric(1:(model.C), model.migration_rates[pid, :])
    m = rand(model.rng, d)
    if m ≠ pid
        move_agent!(agent, m, model)
    end
end

migrate! (generic function with 1 method)

In [10]:
function transmit!(agent, model)
    agent.status == :S && return
    rate = if agent.days_infected < model.detection_time
        model.β_und[agent.pos]
    else
        model.β_det[agent.pos]
    end

    d = Poisson(rate)
    n = rand(model.rng, d)
    n == 0 && return

    for contactID in ids_in_position(agent, model)
        contact = model[contactID]
        if contact.status == :S ||
           (contact.status == :R && rand(model.rng) ≤ model.reinfection_probability)
            contact.status = :I
            n -= 1
            n == 0 && return
        end
    end
end

transmit! (generic function with 1 method)

In [11]:
update!(agent, model) = agent.status == :I && (agent.days_infected += 1)

update! (generic function with 1 method)

In [13]:
function recover_or_die!(agent, model)
    if agent.days_infected ≥ model.infection_period
        if rand(model.rng) ≤ model.death_rate
            kill_agent!(agent, model)
        else
            agent.status = :R
            agent.days_infected = 0
        end
    end
end

recover_or_die! (generic function with 1 method)

In [14]:
function agent_step!(agent::PoorSoul, model)
    migrate!(agent, model)
    transmit!(agent, model)
    update!(agent, model)
    recover_or_die!(agent, model)
end

agent_step! (generic function with 1 method)

In [15]:
model = make_SIRgraph(; SIRgraphparams...)

AgentBasedModel with 19520 agents of type PoorSoul
 space: GraphSpace with 8 positions and 56 edges
 scheduler: fastest
 properties: Ns, Is, β_und, β_det, migration_rates, infection_period, reinfection_probability, detection_time, C, death_rate

In [16]:
anim = @animate for i in 1:40
    Agents.step!(model, agent_step!, 1)


    cityPops = length.(model.space.s)
    xs = 1:length(cityPops)
    infected = map(x -> count(model[id].status == :I for id in x), model.space.s)
    infectedTotal = sum(infected)


    pl = bar(xs, cityPops, label="Total")
    bar!(pl, xs, infected, label="Infected", 
         xlabel="City", ylabel="Population",
         title = "Step $i: $infectedTotal infected"
    )
end

Animation("C:\\Users\\SOSIRI~1\\AppData\\Local\\Temp\\jl_4ZoKLQ", ["000001.png", "000002.png", "000003.png", "000004.png", "000005.png", "000006.png", "000007.png", "000008.png", "000009.png", "000010.png"  …  "000031.png", "000032.png", "000033.png", "000034.png", "000035.png", "000036.png", "000037.png", "000038.png", "000039.png", "000040.png"])

In [17]:
mp4(anim, fps = 5)

┌ Info: Saved animation to 
│   fn = D:\github\BEBI-5009\julia\abm\tmp.mp4
└ @ Plots C:\Users\sosiristseng\.julia\packages\Plots\OchXa\src\animation.jl:104


Plots.AnimatedGif("D:\\github\\BEBI-5009\\julia\\abm\\tmp.mp4")